# This notebook provides interactive widgets to play with values for the SKA1 SDP

## First, do the necessary set-up

In [1]:
'''
The following commands import the python modules, methods and code fragments that are required to run this notebook. 
Please refer to the following python files for implementation details (omitted here for readability)
* env_setup.py -- code for setting up the python environment
* parameter_definitions.py -- contains definitions of variables, primary telescope parameters  
* formulae.py -- contains formulae for defiving secondary telescope-specific parameters from input parameters
* implementation.py -- contains methods for performing computations (i.e. crunching the numbers)
'''

from IPython.html.widgets import interact, interactive, fixed
from IPython.html.widgets.interaction import interact_manual
from IPython.html import widgets
from IPython.display import clear_output, display, HTML

from env_setup import *
from parameter_definitions import *
from formulae import *
from implementation import *

def show_table(header, titles, values, units):
    s = '<h3>%s:</h3><table>\n' % header
    assert len(titles) == len(values)
    assert len(titles) == len(units)
    for i in range(len(titles)):
        s += '<tr><td>{0}</td><td><font color="blue">{1}</font> {2}</td></tr>\n'.format(titles[i], values[i], units[i])
    s += '</table>'
    display(HTML(s))
    
band_lookup = {'SKA1-Low':'Low', 'SKA1-Mid (Band 1)':'Mid1', 'SKA1-Survey (Band 1)':'Sur1'}
telescopes_pretty_print =('SKA1-Low', 'SKA1-Mid (Band 1)', 'SKA1-Survey (Band 1)')
modes_pretty_print = ('Continuum', 'Spectral', 'SlowTrans')

mode_lookup = {}
for key in modes_pretty_print:
    mode_lookup[key] = key

def compute_results(tp, Tsnap=None, Nfacet=None):
    """
    Computes the output parameters that we are interested in. Tsnap and Nfacet may either be specified, or omitted.
    If omitted, Tsnap and Nfacet are automtically computed via numerical optimization.
    """
    # NB: The line below defines the output variables to be computed
    expressions = (tp.Mbuf_vis, tp.Mw_cache/1.0e12, tp.Npix_linear, tp.Rio*1e3, tp.Rflop)
    result_value_string = []

    if Tsnap is None and Nfacet is None:
        (Tsnap, Nfacet) = find_optimal_Tsnap_Nfacet(tp, verbose=True)
        result_value_string.append('%d' % Nfacet)
        result_value_string.append('%.3g' % Tsnap)
    
    for expression in expressions:
        expression_subst = expression.subs({tp.Tsnap : Tsnap, tp.Nfacet : Nfacet})
        result = evaluate_binned_expression(expression_subst, tp)
        if expression is not tp.Npix_linear: 
            result_value_string.append('%.3g' % result)
        else:
            result_value_string.append('%d' % result)

    return result_value_string
    
def compute_flops(max_baseline, Nfacet, Tsnap, Telescope, Mode):    
    band = band_lookup[Telescope]
    titles = ('Max Baseline', 'Telescope', 'Band', 'Mode', 'Tsnap', 'Nfacet')
    values = (max_baseline, Telescope, band, Mode, Tsnap, Nfacet)
    units = ('km', '', '', '', 'sec', '')
    show_table('Arguments', titles, values, units)
    
    result_titles = ('Visibility Buffer', 'Working (cache) memory', 'Image side length', 'I/O Rate', 'Total Compute Requirement',)
    result_units = ('PetaBytes', 'TeraBytes', 'pixels', 'TeraBytes/s','PetaFLOPS',)
    
    tp = calc_tel_params(band=band, mode=mode_lookup[Mode])  # Telescope parameters
    max_allowed_baseline = tp.baseline_bins[-1] / u.km
    
    if max_baseline <= max_allowed_baseline:
        tp.Bmax = max_baseline * u.km
        result_value_string = compute_results(tp, Tsnap, Nfacet)
        show_table('Computed Values', result_titles, result_value_string, result_units)
    else :
        msg = 'ERROR: max_baseline exceeds the maximum allowed baseline of %g km for this telescope.' % max_allowed_baseline
        s = '<font color="red"><b>{0}</b>.<br>Adjust to recompute.</font>'.format(msg)
        display(HTML(s))

def compute_flops_auto(max_baseline, Telescope, Mode):    
    band = band_lookup[Telescope]
    titles = ('Max Baseline', 'Telescope', 'Band', 'Mode')
    values = (max_baseline, Telescope, band, Mode)
    units = ('km', '', '', '')
    show_table('Arguments', titles, values, units)
    display(HTML('<font color="blue">Computing result -- this may take several (tens of) seconds.</font>'))
    
    result_titles = ('Number of Facets', 'Snapshot Time', 'Visibility Buffer', 'Working (cache) memory', 'Image side length', 'I/O Rate', 'Total Compute Requirement',)
    result_units = ('', 'sec.', 'PetaBytes', 'TeraBytes', 'pixels', 'TeraBytes/s','PetaFLOPS',)
    
    tp = calc_tel_params(band=band, mode=mode_lookup[Mode])  # Telescope parameters
    max_allowed_baseline = tp.baseline_bins[-1] / u.km
    
    if max_baseline <= max_allowed_baseline:
        tp.Bmax = max_baseline * u.km
        result_value_string = compute_results(tp)
        show_table('Computed Values', result_titles, result_value_string, result_units)
    else :
        msg = 'ERROR: max_baseline exceeds the maximum allowed baseline of %g km for this telescope.' % max_allowed_baseline
        s = '<font color="red"><b>{0}</b>.<br>Adjust to recompute.</font>'.format(msg)
        display(HTML(s))

IPython console for SymPy 0.7.6 (Python 2.7.7-32-bit) (ground types: python)


:0: FutureWarning: IPython widgets are experimental and may change in the future.


## Now, we can interactively play with parameters

### The first option is automatic updating of results as the sliders are moved. This may be sluggish

In [2]:
interact(compute_flops, max_baseline=(10,200), Nfacet=(1,10,1), Tsnap=(1.2,1800), Telescope=telescopes_pretty_print, 
         Mode=modes_pretty_print);

Max Baseline,78 km
Telescope,SKA1-Low
Band,Low
Mode,Continuum
Tsnap,900.6 sec
Nfacet,5


Visibility Buffer,0.67 PetaBytes
Working (cache) memory,0.501 TeraBytes
Image side length,75237 pixels
I/O Rate,4.27 TeraBytes/s
Total Compute Requirement,10.1 PetaFLOPS


### The second option is manual triggering of recompute events (recommended). 

**This allows more conveniently computing elaborate (slow) visualizations per computation**

In [3]:
# In this example, Tsnap and Nfacet are *automatically* chosen so as to minimize the value of Rflop

interact_manual(compute_flops_auto, max_baseline=(10,200), Telescope=telescopes_pretty_print, 
         Mode=modes_pretty_print);

Max Baseline,99 km
Telescope,SKA1-Mid (Band 1)
Band,Mid1
Mode,Continuum


Evaluating Nfacets = 1
Tsnap has been optimized as : 143.974851, yielding a minimum value of 21.708558 Peta-units
Evaluating Nfacets = 2
Tsnap has been optimized as : 275.301183, yielding a minimum value of 11.125546 Peta-units
Evaluating Nfacets = 3
Tsnap has been optimized as : 395.012650, yielding a minimum value of 7.768829 Peta-units
Evaluating Nfacets = 4
Tsnap has been optimized as : 506.951489, yielding a minimum value of 6.148998 Peta-units
Evaluating Nfacets = 5
Tsnap has been optimized as : 613.150575, yielding a minimum value of 5.220664 Peta-units
Evaluating Nfacets = 6
Tsnap has been optimized as : 714.840884, yielding a minimum value of 4.641366 Peta-units
Evaluating Nfacets = 7
Tsnap has been optimized as : 812.823718, yielding a minimum value of 4.265396 Peta-units
Evaluating Nfacets = 8
Tsnap has been optimized as : 907.602562, yielding a minimum value of 4.021690 Peta-units
Evaluating Nfacets = 9
Tsnap has been optimized as : 999.684889, yielding a minimum value of 3

Number of Facets,11
Snapshot Time,1.18e+03 sec.
Visibility Buffer,0.106 PetaBytes
Working (cache) memory,0.0256 TeraBytes
Image side length,159594 pixels
I/O Rate,3.27 TeraBytes/s
Total Compute Requirement,3.75 PetaFLOPS
